#### Importando as dependências

In [1]:
import pandas as pd
from sqlalchemy import create_engine, exc
import requests
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc, html

#### Baixando e carregando os dados brutos na camada raw do postgres

In [2]:
# URL do arquivo CSV

url = 'https://dadosabertos.mec.gov.br/images/conteudo/prouni/2020/ProuniRelatorioDadosAbertos2020.csv'
file_path = 'ProuniRelatorioDadosAbertos2020.csv'

# Download do arquivo CSV
try:
    response = requests.get(url)
    response.raise_for_status()  # Verifica se o download foi bem-sucedido
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print("Arquivo CSV baixado com sucesso!")
except requests.HTTPError as e:
    print(f"Erro ao baixar o arquivo: {e}")
except Exception as e:
    print(f"Ocorreu um erro inesperado ao baixar o arquivo: {e}")

# Leitura do arquivo CSV com especificação de codificação e delimitador
try:
    data = pd.read_csv(file_path, encoding='latin1', sep=';')
    print("Arquivo CSV lido com sucesso!")
except pd.errors.ParserError as e:
    print(f"Erro ao ler o CSV: {e}")
except Exception as e:
    print(f"Ocorreu um erro inesperado ao ler o arquivo: {e}")

# Conexão com o banco de dados PostgreSQL
try:
    user = 'postgres'
    password = '0000'
    host = 'localhost'
    port = '5432'
    database = 'raw_data'
    
    engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

    # Carregando os dados no banco de dados, sobrescrevendo a tabela se ela existir
    data.to_sql('raw_data_table', con=engine, if_exists='replace', index=False)
    print("Dados carregados com sucesso!")
except exc.SQLAlchemyError as e:
    print(f"Erro ao conectar ou enviar dados para o banco de dados: {e}")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

# O resultado demora em média de 1 a 4 minutos

Arquivo CSV baixado com sucesso!
Arquivo CSV lido com sucesso!
Dados carregados com sucesso!


#### Lendo os dados da camada raw do postgres

In [4]:
# Configurações de conexão com o banco de dados PostgreSQL
user = 'postgres'
password = '0000'
host = 'localhost'
port = '5432'
database = 'raw_data'

# Criação do engine de conexão com o banco de dados
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

# Leitura dos dados da tabela no banco de dados
query = "SELECT * FROM raw_data_table"  # Substitua 'your_table_name' pelo nome da tabela
data = pd.read_sql_query(query, con=engine)

# Removendo linhas com valores nulos e criando uma cópia para evitar o aviso
data_cleaned = data.dropna().copy()

# Normalizando colunas de texto (convertendo para minúsculas)
cols_to_normalize = ['NOME_IES_BOLSA', 'MUNICIPIO', 'CAMPUS', 'TIPO_BOLSA',
                     'MODALIDADE_ENSINO_BOLSA', 'NOME_CURSO_BOLSA', 'NOME_TURNO_CURSO_BOLSA',
                     'SEXO_BENEFICIARIO', 'RACA_BENEFICIARIO', 'REGIAO_BENEFICIARIO',
                     'UF_BENEFICIARIO', 'MUNICIPIO_BENEFICIARIO', 'BENEFICIARIO_DEFICIENTE_FISICO']

# Usando .loc para evitar SettingWithCopyWarning
for col in cols_to_normalize:
    data_cleaned.loc[:, col] = data_cleaned[col].str.lower()

# Convertendo datas para formato datetime
data_cleaned.loc[:, 'DATA_NASCIMENTO'] = pd.to_datetime(data_cleaned['DATA_NASCIMENTO'], format='%d/%m/%Y', errors='coerce')

# Conexão com o banco de dados PostgreSQL para salvar os dados transformados
silver_database = 'silver_data'
engine_silver = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{silver_database}')

# Carregando os dados Limpos no banco de dados, sobrescrevendo a tabela se ela existir
data_cleaned.to_sql('silver_data_table', con=engine_silver, if_exists='replace', index=False)

print("Dados transformados e carregados com sucesso!")

Dados transformados e carregados com sucesso!


#### Lê os dados da camada silver no postgres e fazer suas repectivas análises

In [5]:
# Conexão com o banco de dados PostgreSQL para leitura dos dados da camada silver
user = 'postgres'
password = '0000'
host = 'localhost'
port = '5432'
database_silver = 'silver_data'

engine_silver = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database_silver}')

# Consulta dos dados da tabela na camada silver
data_cleaned = pd.read_sql_table('silver_data_table', con=engine_silver)

# Análises e criação de DataFrames

# Análise 1: Distribuição por Sexo e Raça
sexo_raca_distribution = data_cleaned.groupby(['SEXO_BENEFICIARIO', 'RACA_BENEFICIARIO']).size().reset_index(name='count')

# Análise 2: Distribuição Etária
data_cleaned['IDADE'] = data_cleaned['DATA_NASCIMENTO'].apply(lambda x: pd.Timestamp.now().year - x.year if pd.notnull(x) else None)
age_distribution = data_cleaned['IDADE'].value_counts().reset_index(name='count').rename(columns={'index': 'IDADE'})

# Análise 3: Distribuição Regional e Estadual
regional_state_distribution = data_cleaned.groupby(['REGIAO_BENEFICIARIO', 'UF_BENEFICIARIO']).size().reset_index(name='count')

# Análise 4: Instituições de Ensino
institutions_distribution = data_cleaned['NOME_IES_BOLSA'].value_counts().reset_index(name='count').rename(columns={'index': 'NOME_IES_BOLSA'})

# Análise 5: Cursos Mais Populares
courses_distribution = data_cleaned['NOME_CURSO_BOLSA'].value_counts().reset_index(name='count').rename(columns={'index': 'NOME_CURSO_BOLSA'})

# Análise 6: Tipos de Bolsas e Modalidades
bolsas_modalidades_distribution = data_cleaned.groupby(['TIPO_BOLSA', 'MODALIDADE_ENSINO_BOLSA']).size().reset_index(name='count')

# Conexão com o banco de dados PostgreSQL para salvar os dados na camada gold
database_gold = 'gold_data'
engine_gold = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database_gold}')

# Função para carregar dados no banco de dados gold com tratamento de erro
def load_data_to_db(df, table_name, engine):
    try:
        with engine.connect() as connection:
            df.to_sql(table_name, con=connection, if_exists='replace', index=False)
            print(f"Dados carregados com sucesso na tabela {table_name}!")
    except exc.SQLAlchemyError as e:
        print(f"Erro ao carregar dados na tabela {table_name}: {e}")
    except Exception as e:
        print(f"Erro inesperado ao carregar dados na tabela {table_name}: {e}")

# Carregar as tabelas no banco de dados gold com tratamento de erro
load_data_to_db(sexo_raca_distribution, 'sexo_raca_distribution', engine_gold)
load_data_to_db(age_distribution, 'age_distribution', engine_gold)
load_data_to_db(regional_state_distribution, 'regional_state_distribution', engine_gold)
load_data_to_db(institutions_distribution, 'institutions_distribution', engine_gold)
load_data_to_db(courses_distribution, 'courses_distribution', engine_gold)
load_data_to_db(bolsas_modalidades_distribution, 'bolsas_modalidades_distribution', engine_gold)


Dados carregados com sucesso na tabela sexo_raca_distribution!
Dados carregados com sucesso na tabela age_distribution!
Dados carregados com sucesso na tabela regional_state_distribution!
Dados carregados com sucesso na tabela institutions_distribution!
Dados carregados com sucesso na tabela courses_distribution!
Dados carregados com sucesso na tabela bolsas_modalidades_distribution!


In [6]:
# Conexão com o banco de dados PostgreSQL gold_data
user = 'postgres'
password = '0000'
host = 'localhost'
port = '5432'
database_gold = 'gold_data'
engine_gold = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database_gold}')

# Consulta os dados das tabelas e extraindo para variáveis

sexo_raca_distribution = pd.read_sql_table('sexo_raca_distribution', con=engine_gold)

age_distribution = pd.read_sql_table('age_distribution', con=engine_gold)

regional_state_distribution = pd.read_sql_table('regional_state_distribution', con=engine_gold)

institutions_distribution = pd.read_sql_table('institutions_distribution', con=engine_gold)

courses_distribution = pd.read_sql_table('courses_distribution', con=engine_gold)

bolsas_modalidades_distribution = pd.read_sql_table('bolsas_modalidades_distribution', con=engine_gold)

In [7]:
# Gráfico 1: Distribuição por Sexo e Raça,  Um gráfico de barras que mostra a contagem de beneficiários por sexo e raça.
fig1 = px.bar(sexo_raca_distribution, x='SEXO_BENEFICIARIO', y='count', color='RACA_BENEFICIARIO',
              title='DISTRIBUIÇÃO POR SEXO E RAÇA')
fig1.show()

In [8]:
# Gráfico 2: Distribuição Etária, Um histograma que mostra a distribuição de idades dos beneficiários.
fig2 = px.histogram(age_distribution, x='IDADE', y='count', nbins=20, title='DISTRIBUIÇÃO ETÁRIA')
fig2.show()

In [9]:
# Gráfico 3: Distribuição Regional e Estadual, Um gráfico de barras que mostra a contagem de beneficiários por região e estado.
fig3 = px.bar(regional_state_distribution, x='REGIAO_BENEFICIARIO', y='count', color='UF_BENEFICIARIO',
              title='DISTRIBUIÇÃO REGIONAL E ESTADUAL')
fig3.show()

In [10]:
# Gráfico 4: Instituições de Ensino (limitado a 70 instituições)

# Seleciona as 70 instituições com mais bolsas
top_70_institutions = institutions_distribution.nlargest(70, 'count')

fig4 = px.bar(top_70_institutions, x='NOME_IES_BOLSA', y='count',
              title='TOP 70 INSTITUIÇÕES DE ENSINO COM BOLSAS', labels={'count': 'Número de Bolsas'})
fig4.update_layout(xaxis={'categoryorder':'total descending'})  # Ordena por número de bolsas
fig4.show()

In [11]:
# Gráfico 5: Cursos Mais Populares (limitado a 70 cursos para melhor visualização), Um gráfico de barras que mostra os cursos mais populares com base no número de bolsas.
top_70_courses = courses_distribution.nlargest(70, 'count')
fig5 = px.bar(top_70_courses, x='NOME_CURSO_BOLSA', y='count',
              title='TOP 70 CURSOS MAIS POPULARES', labels={'count': 'Número de Bolsas'})
fig5.update_layout(xaxis={'categoryorder':'total descending'})
fig5.show()

In [12]:
# Gráfico 6: Tipos de Bolsas e Modalidades,  Um gráfico de barras que mostra a distribuição de tipos de bolsas e modalidades de ensino.
fig6 = px.bar(bolsas_modalidades_distribution, x='TIPO_BOLSA', y='count', color='MODALIDADE_ENSINO_BOLSA',
              title='TIPOS DE BOLSAS E MODALIDADES')
fig6.show()

In [13]:
# Supondo que as figuras (fig1, fig2, ..., fig6) já estejam definidas no mesmo escopo ou importadas corretamente
# Exemplo de definições (isso precisa ser ajustado conforme suas figuras reais):
# fig1, fig2, fig3, fig4, fig5, fig6 = [some_plotly_figure] * 6

# Inicializa a aplicação Dash
app = dash.Dash(__name__)

# Define o layout da aplicação
app.layout = html.Div([
    html.H1("Dashboard de Análises de Bolsas"),

    # Gráficos exibidos em sequência
    dcc.Graph(figure=fig1, id='graph-1'),
    dcc.Graph(figure=fig2, id='graph-2'),
    dcc.Graph(figure=fig3, id='graph-3'),
    dcc.Graph(figure=fig4, id='graph-4'),
    dcc.Graph(figure=fig5, id='graph-5'),
    dcc.Graph(figure=fig6, id='graph-6')
])

# Não há necessidade de callback já que todos os gráficos são renderizados de uma vez

# Executa a aplicação
if __name__ == '__main__':
    app.run_server(host='127.0.0.1', port=8050, debug=True)
    print("Acesse o seu dashboard em http://127.0.0.1:8050/")


Acesse o seu dashboard em http://127.0.0.1:8050/
